# Importing Necessary Packages

In [30]:
# Importing all the functions I made for data creation/manipulation
from data_creation import data_creation, pickle_me, get_pickle

# Model Building
import pandas as pd
import numpy as np

# Neural Network Building
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import regularizers
from keras.layers import Dense, Flatten, Dropout, Conv1D, Conv2D, ELU

# Classification Metrics
from sklearn.metrics import confusion_matrix
from data_creation import get_precisions
from sklearn.metrics import precision_score, recall_score, f1_score

# Packaging Data
import pickle

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Data Creation

#### I only need to run this once to create my data. It will pickle my data for future reference.
I will only run the categories object since I call on it later in the notebook.

In [2]:
# Directory where all the images are stored.
#data_dir = 'C://Users/Cristian/Documents/flatiron/Capstone/Fruit'

# Categories of Fruit with the correct labels.
categories = ['Apple', 'Banana', 'Carambola', 'Guava', 'Kiwi', 'Mango',
              'Muskmelon', 'Orange', 'Peach', 'Pear', 'Persimmon', 'Pitaya',
              'Plum', 'Pomegranate', 'Tomato']

##### Creating Grayscale data and pickling it

In [3]:
#X_gray, y_gray = data_creation(50, 0, categories, data_dir)
#pickle_me(X_gray,y_gray, 'X_grayscale', 'y_grayscale')

##### Creating Color data and pickling it

In [4]:
#X_color, y_color = data_creation(50, 1, categories, data_dir)
#pickle_me(X_color, y_color,'X_color', 'y_color')

##### Getting our pickled data for grayscale images and color images

In [5]:
X_grayscale, y_grayscale = get_pickle('X_grayscale.pickle', 'y_grayscale.pickle')

In [6]:
X_rgb, y_rgb = get_pickle('X_color.pickle', 'y_color.pickle')

# Data Manipulation

#### Creating a train, test split for cross-validation.
'gs' will be for grayscale images, 'rgb' will be for colore images

In [7]:
X_gs_train, X_gs_test, y_gs_train, y_gs_test = train_test_split(X_grayscale,
                                                                y_grayscale, 
                                                                test_size=.2,
                                                                random_state=42)

In [8]:
X_rgb_train, X_rgb_test, y_rgb_train, y_rgb_test = train_test_split(X_rgb,
                                                                    y_rgb, 
                                                                    test_size=.2,
                                                                    random_state=42)

#### Scaling our data and reshaping so they are the correct input for our models.

In [9]:
X_gs_train_sc = X_gs_train/255.0
X_gs_test_sc = X_gs_test/255.0

X_gs_train_reshape = X_gs_train_sc.reshape((X_gs_train.shape[0],50,50,))
X_gs_test_reshape = X_gs_test_sc.reshape((X_gs_test.shape[0],50,50,))

In [10]:
X_rgb_train_sc = X_rgb_train/255.0
X_rgb_test_sc = X_rgb_test/255.0

X_rgb_train_reshape = X_rgb_train_sc.reshape((X_rgb_train.shape[0],50,50,3))
X_rgb_test_reshape = X_rgb_test_sc.reshape((X_rgb_test.shape[0],50,50,3))

#### For our models we need a target input of (15,) instead of (1,), which is the number of fruit labels we have

In [11]:
y_gs_train_dummies = pd.get_dummies(y_gs_train)
y_gs_test_dummies = pd.get_dummies(y_gs_test)

In [12]:
y_rgb_train_dummies = np.array(pd.get_dummies(y_rgb_train))
y_rgb_test_dummies = np.array(pd.get_dummies(y_rgb_test))

# Model Building
As a way to better visualize how models are doing compared to one another, I will create the neural networks, compile them, and fit our data. Then, I will have them predict classes and put our classification metrics into a dataframe.

## Grayscale Models

### Model 1
First Simple Model

In [13]:
model_1 = Sequential([Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_1.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy', 'mae', 'mse'])
model_1_results = model_1.fit(X_gs_train_reshape, y_gs_train_dummies, epochs=5, validation_data =(X_gs_test_reshape, y_gs_test_dummies))


Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 10s 273us/step - loss: 1.6822 - accuracy: 0.5513 - mae: 0.0917 - mse: 0.0420 - val_loss: 1.5720 - val_accuracy: 0.5347 - val_mae: 0.0836 - val_mse: 0.0409
Epoch 2/5
35524/35524 [==============================] - 2s 55us/step - loss: 1.2086 - accuracy: 0.6879 - mae: 0.0733 - mse: 0.0310 - val_loss: 1.3397 - val_accuracy: 0.5903 - val_mae: 0.0751 - val_mse: 0.0362
Epoch 3/5
35524/35524 [==============================] - 2s 55us/step - loss: 1.1017 - accuracy: 0.7126 - mae: 0.0668 - mse: 0.0279 - val_loss: 1.1192 - val_accuracy: 0.7002 - val_mae: 0.0658 - val_mse: 0.0284
Epoch 4/5
35524/35524 [==============================] - 2s 54us/step - loss: 1.0426 - accuracy: 0.7258 - mae: 0.0630 - mse: 0.0261 - val_loss: 1.0761 - val_accuracy: 0.7228 - val_mae: 0.0630 - val_mse: 0.0268
Epoch 5/5
35524/35524 [==============================] - 2s 53us/step - loss: 1.0030 - accuracy: 0.7350 - ma

### Model 2
Adding a convolutional layer

In [14]:
model_2 = Sequential([Conv1D(100, kernel_size=(1), activation='relu', input_shape=(50,50)),
                      Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_2.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy', 'mae', 'mse'])
model_2_results = model_2.fit(X_gs_train_reshape, y_gs_train_dummies, epochs=5, validation_data =(X_gs_test_reshape, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 4s 123us/step - loss: 1.1140 - accuracy: 0.6936 - mae: 0.0634 - mse: 0.0278 - val_loss: 1.2451 - val_accuracy: 0.6747 - val_mae: 0.0605 - val_mse: 0.0322
Epoch 2/5
35524/35524 [==============================] - 4s 115us/step - loss: 0.7981 - accuracy: 0.7773 - mae: 0.0463 - mse: 0.0198 - val_loss: 0.9840 - val_accuracy: 0.7204 - val_mae: 0.0510 - val_mse: 0.0254
Epoch 3/5
35524/35524 [==============================] - 4s 121us/step - loss: 0.7315 - accuracy: 0.7915 - mae: 0.0423 - mse: 0.0182 - val_loss: 0.8898 - val_accuracy: 0.7378 - val_mae: 0.0479 - val_mse: 0.0226
Epoch 4/5
35524/35524 [==============================] - 4s 118us/step - loss: 0.6933 - accuracy: 0.7988 - mae: 0.0401 - mse: 0.0173 - val_loss: 0.7632 - val_accuracy: 0.7802 - val_mae: 0.0418 - val_mse: 0.0187
Epoch 5/5
35524/35524 [==============================] - 4s 114us/step - loss: 0.6662 - accuracy: 0.8058 - 

### Model 3
Changing the number of neurons for the convolutional layer

In [15]:
model_3 = Sequential([Conv1D(50, kernel_size=(1), activation='relu', input_shape=(50,50)),
                      Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_3.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy', 'mae', 'mse'])
model_3_results = model_3.fit(X_gs_train_reshape, y_gs_train_dummies, epochs=5, validation_data =(X_gs_test_reshape, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 4s 118us/step - loss: 1.2267 - accuracy: 0.6641 - mae: 0.0703 - mse: 0.0309 - val_loss: 1.0069 - val_accuracy: 0.7358 - val_mae: 0.0560 - val_mse: 0.0252
Epoch 2/5
35524/35524 [==============================] - 4s 117us/step - loss: 0.8974 - accuracy: 0.7544 - mae: 0.0524 - mse: 0.0223 - val_loss: 0.9587 - val_accuracy: 0.7216 - val_mae: 0.0538 - val_mse: 0.0241
Epoch 3/5
35524/35524 [==============================] - 4s 118us/step - loss: 0.8248 - accuracy: 0.7711 - mae: 0.0477 - mse: 0.0204 - val_loss: 0.8582 - val_accuracy: 0.7623 - val_mae: 0.0474 - val_mse: 0.0210
Epoch 4/5
35524/35524 [==============================] - 4s 117us/step - loss: 0.7837 - accuracy: 0.7779 - mae: 0.0452 - mse: 0.0195 - val_loss: 0.8519 - val_accuracy: 0.7591 - val_mae: 0.0462 - val_mse: 0.0209
Epoch 5/5
35524/35524 [==============================] - 4s 118us/step - loss: 0.7564 - accuracy: 0.7848 - 

### Model 4
Adding another dense layer

In [16]:
model_4 = Sequential([Conv1D(50, kernel_size=(1), activation='relu', input_shape=(50,50)),
                      Flatten(input_shape=(50,50)),
                      Dense(30, activation='sigmoid'),
                      Dense(15, activation='softmax')])
model_4.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy', 'mae', 'mse'])
model_4_results = model_4.fit(X_gs_train_reshape, y_gs_train_dummies,epochs=5, validation_data = (X_gs_test_reshape, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 5s 129us/step - loss: 1.6375 - accuracy: 0.5964 - mae: 0.0972 - mse: 0.0426 - val_loss: 1.3698 - val_accuracy: 0.6863 - val_mae: 0.0861 - val_mse: 0.0356
Epoch 2/5
35524/35524 [==============================] - 4s 125us/step - loss: 1.1973 - accuracy: 0.7187 - mae: 0.0762 - mse: 0.0305 - val_loss: 1.1637 - val_accuracy: 0.7189 - val_mae: 0.0731 - val_mse: 0.0295
Epoch 3/5
35524/35524 [==============================] - 4s 126us/step - loss: 1.0578 - accuracy: 0.7406 - mae: 0.0669 - mse: 0.0264 - val_loss: 1.0966 - val_accuracy: 0.7078 - val_mae: 0.0674 - val_mse: 0.0276
Epoch 4/5
35524/35524 [==============================] - 4s 124us/step - loss: 0.9772 - accuracy: 0.7526 - mae: 0.0614 - mse: 0.0241 - val_loss: 0.9633 - val_accuracy: 0.7499 - val_mae: 0.0595 - val_mse: 0.0236
Epoch 5/5
35524/35524 [==============================] - 4s 121us/step - loss: 0.9208 - accuracy: 0.7615 - 

### Model 5
Adding a dropout layer

In [17]:
model_5 = Sequential([Conv1D(50, kernel_size=(1), activation='relu', input_shape=(50,50)),
                      Flatten(input_shape=(50,50)),
                      Dropout(0.2),
                      Dense(30, activation='sigmoid'),
                      Dense(15, activation='softmax')])
model_5.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy', 'mae', 'mse'])
model_5_results = model_5.fit(X_gs_train_reshape, y_gs_train_dummies,epochs=5, validation_data = (X_gs_test_reshape, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 6s 161us/step - loss: 1.6911 - accuracy: 0.5686 - mae: 0.0988 - mse: 0.0439 - val_loss: 1.3878 - val_accuracy: 0.6726 - val_mae: 0.0872 - val_mse: 0.0364
Epoch 2/5
35524/35524 [==============================] - 6s 167us/step - loss: 1.2577 - accuracy: 0.6988 - mae: 0.0796 - mse: 0.0325 - val_loss: 1.1640 - val_accuracy: 0.7094 - val_mae: 0.0741 - val_mse: 0.0298
Epoch 3/5
35524/35524 [==============================] - 6s 164us/step - loss: 1.1148 - accuracy: 0.7250 - mae: 0.0708 - mse: 0.0283 - val_loss: 1.0637 - val_accuracy: 0.7304 - val_mae: 0.0673 - val_mse: 0.0268
Epoch 4/5
35524/35524 [==============================] - 5s 152us/step - loss: 1.0330 - accuracy: 0.7368 - mae: 0.0652 - mse: 0.0259 - val_loss: 1.0148 - val_accuracy: 0.7364 - val_mae: 0.0632 - val_mse: 0.0253
Epoch 5/5
35524/35524 [==============================] - 5s 154us/step - loss: 0.9789 - accuracy: 0.7461 - 

## Color Images Model

### Model 6
We are starting to use color images since color is a big factor in determing fruit apart. 

Model 6 showed a loss in accuracy, mase, and mse after 4 epochs, so it was lowered to 4 epochs exactly.

In [99]:
model_6 = Sequential([Conv2D(100, kernel_size=(1), activation='relu'),
                      Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_6.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy', 'mae', 'mse'])
model_6_results = model_6.fit(X_rgb_train_reshape, y_rgb_train_dummies, epochs=4, validation_data=(X_rgb_test_reshape, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/4
35524/35524 [==============================] - 104s 3ms/step - loss: 1.0420 - accuracy: 0.8480 - mae: 0.0311 - mse: 0.0149 - val_loss: 0.5346 - val_accuracy: 0.8249 - val_mae: 0.0295 - val_mse: 0.0176
Epoch 2/4
35524/35524 [==============================] - 105s 3ms/step - loss: 0.1935 - accuracy: 0.9412 - mae: 0.0154 - mse: 0.0060 - val_loss: 0.2376 - val_accuracy: 0.9205 - val_mae: 0.0170 - val_mse: 0.0076
Epoch 3/4
35524/35524 [==============================] - 103s 3ms/step - loss: 0.1403 - accuracy: 0.9606 - mae: 0.0119 - mse: 0.0043 - val_loss: 0.2398 - val_accuracy: 0.9168 - val_mae: 0.0162 - val_mse: 0.0080
Epoch 4/4
35524/35524 [==============================] - 104s 3ms/step - loss: 0.1126 - accuracy: 0.9697 - mae: 0.0100 - mse: 0.0034 - val_loss: 0.2754 - val_accuracy: 0.9108 - val_mae: 0.0169 - val_mse: 0.0086


### Model 7

Changing the activation function to see if there is a difference

In [19]:
model_7 = Sequential([Conv2D(50, kernel_size=(1), activation='sigmoid'),
                      Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_7.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy', 'mae', 'mse'])
model_7_results = model_7.fit(X_rgb_train_reshape, y_rgb_train_dummies, epochs=5, validation_data=(X_rgb_test_reshape, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 55s 2ms/step - loss: 48.8626 - accuracy: 0.1039 - mae: 0.1196 - mse: 0.1073 - val_loss: 26.4535 - val_accuracy: 0.0688 - val_mae: 0.1238 - val_mse: 0.1192
Epoch 2/5
35524/35524 [==============================] - 59s 2ms/step - loss: 7.3643 - accuracy: 0.2447 - mae: 0.1032 - mse: 0.0778 - val_loss: 11.1094 - val_accuracy: 0.1575 - val_mae: 0.1138 - val_mse: 0.0899
Epoch 3/5
35524/35524 [==============================] - 55s 2ms/step - loss: 3.5422 - accuracy: 0.3783 - mae: 0.0882 - mse: 0.0595 - val_loss: 4.3879 - val_accuracy: 0.1672 - val_mae: 0.1089 - val_mse: 0.0893
Epoch 4/5
35524/35524 [==============================] - 55s 2ms/step - loss: 2.2532 - accuracy: 0.4894 - mae: 0.0768 - mse: 0.0472 - val_loss: 7.1273 - val_accuracy: 0.1425 - val_mae: 0.1137 - val_mse: 0.0985
Epoch 5/5
35524/35524 [==============================] - 54s 2ms/step - loss: 1.8397 - accuracy: 0.5430 - ma

### Model 8

Adding a dropout layer

In [53]:
model_8 = Sequential([Conv2D(100, kernel_size=(1), activation='relu'),
                      Flatten(input_shape=(50,50)),
                      Dropout(0.2),
                      Dense(15, activation='softmax')])
model_8.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy', 'mae', 'mse'])
model_8_results = model_8.fit(X_rgb_train_reshape, y_rgb_train_dummies, epochs=5, validation_data=(X_rgb_test_reshape, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 223s 6ms/step - loss: 1.1175 - accuracy: 0.8383 - mae: 0.0341 - mse: 0.0158 - val_loss: 0.5462 - val_accuracy: 0.8151 - val_mae: 0.0337 - val_mse: 0.0175
Epoch 2/5
35524/35524 [==============================] - 221s 6ms/step - loss: 0.2223 - accuracy: 0.9330 - mae: 0.0176 - mse: 0.0068 - val_loss: 0.2631 - val_accuracy: 0.9143 - val_mae: 0.0187 - val_mse: 0.0083
Epoch 3/5
35524/35524 [==============================] - 222s 6ms/step - loss: 0.1661 - accuracy: 0.9519 - mae: 0.0139 - mse: 0.0050 - val_loss: 0.2034 - val_accuracy: 0.9317 - val_mae: 0.0153 - val_mse: 0.0066
Epoch 4/5
35524/35524 [==============================] - 222s 6ms/step - loss: 0.1341 - accuracy: 0.9624 - mae: 0.0117 - mse: 0.0040 - val_loss: 0.2525 - val_accuracy: 0.9194 - val_mae: 0.0167 - val_mse: 0.0080
Epoch 5/5
35524/35524 [==============================] - 221s 6ms/step - loss: 0.1133 - accuracy: 0.9705 - 

#### Model 9
Adding bias regularizers and lowering the neurons for the convolution layer.

In [21]:
model_9 = Sequential([Conv2D(10, kernel_size=(1), activation='relu'),
                      Flatten(input_shape=(50,50)),
                      Dropout(0.2),
                      Dense(64, activation='sigmoid', bias_regularizer=regularizers.l1_l2(l1=0.02,l2=0.02)),
                      Dense(15, activation='softmax')])
model_9.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy', 'mae', 'mse'])
model_9_results = model_9.fit(X_rgb_train_reshape, y_rgb_train_dummies,epochs=5, validation_data = (X_rgb_test_reshape, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 42s 1ms/step - loss: 1.4408 - accuracy: 0.6338 - mae: 0.0893 - mse: 0.0380 - val_loss: 0.8766 - val_accuracy: 0.8056 - val_mae: 0.0649 - val_mse: 0.0236
Epoch 2/5
35524/35524 [==============================] - 42s 1ms/step - loss: 0.7054 - accuracy: 0.8329 - mae: 0.0529 - mse: 0.0187 - val_loss: 0.6421 - val_accuracy: 0.8413 - val_mae: 0.0482 - val_mse: 0.0171
Epoch 3/5
35524/35524 [==============================] - 42s 1ms/step - loss: 0.5562 - accuracy: 0.8602 - mae: 0.0422 - mse: 0.0147 - val_loss: 0.5949 - val_accuracy: 0.8414 - val_mae: 0.0430 - val_mse: 0.0159
Epoch 4/5
35524/35524 [==============================] - 43s 1ms/step - loss: 0.4785 - accuracy: 0.8760 - mae: 0.0366 - mse: 0.0127 - val_loss: 0.4797 - val_accuracy: 0.8697 - val_mae: 0.0354 - val_mse: 0.0129
Epoch 5/5
35524/35524 [==============================] - 42s 1ms/step - loss: 0.4268 - accuracy: 0.8865 - mae: 

### Model 10
Changing the number of neurons in the convolutions layer and using an activity regularizer instead of a bias regularizer.

In [56]:
model_10 = Sequential([Conv2D(85, kernel_size=(1), activation='relu'),
                       Dropout(0.2),
                       Flatten(input_shape=(50,50)),
                       Dense(15, activation='softmax', activity_regularizer=regularizers.l1(0.02))])
model_10.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy', 'mae', 'mse'])
model_10_results = model_10.fit(X_rgb_train_reshape, y_rgb_train_dummies, epochs=5, validation_data=(X_rgb_test_reshape, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 186s 5ms/step - loss: 1.5752 - accuracy: 0.8509 - mae: 0.0326 - mse: 0.0148 - val_loss: 0.9189 - val_accuracy: 0.9138 - val_mae: 0.0213 - val_mse: 0.0085
Epoch 2/5
35524/35524 [==============================] - 185s 5ms/step - loss: 0.8479 - accuracy: 0.9362 - mae: 0.0168 - mse: 0.0064 - val_loss: 0.8814 - val_accuracy: 0.9184 - val_mae: 0.0177 - val_mse: 0.0078
Epoch 3/5
35524/35524 [==============================] - 184s 5ms/step - loss: 0.7962 - accuracy: 0.9536 - mae: 0.0133 - mse: 0.0048 - val_loss: 0.8558 - val_accuracy: 0.9250 - val_mae: 0.0156 - val_mse: 0.0070
Epoch 4/5
35524/35524 [==============================] - 183s 5ms/step - loss: 0.7671 - accuracy: 0.9653 - mae: 0.0112 - mse: 0.0038 - val_loss: 0.8415 - val_accuracy: 0.9308 - val_mae: 0.0144 - val_mse: 0.0067
Epoch 5/5
35524/35524 [==============================] - 186s 5ms/step - loss: 0.7481 - accuracy: 0.9718 - 

# Getting Precisions for our Models

My goal is to get the most amount of correct predictions. This means that of all the predictions I make, I want to get the most amount possible correct. This is similar to the precision metric for classification. Thus, I will obtain the precisions for each fruit for each model. I want to pick the model the has the most correct predictions for each of the classification predictions made.

In [100]:
models = [model_1, model_2, model_3, model_4, model_5,
          model_6, model_7, model_8, model_9, model_10]

In [102]:
precision_df = get_precisions(models, X_gs_test_reshape, y_gs_test, X_rgb_test_reshape, y_rgb_test)

In [103]:
precision_df

,Model_1,Model_2,Model_3,Model_4,Model_5,Model_6,Model_7,Model_8,Model_9,Model_10
Apple,0.789683,0.491741,0.540945,0.417098,0.516704,0.965398,0.947368,0.827430,0.698585,0.914158
Banana,0.423697,0.818182,0.781676,0.490628,0.769231,0.938959,1.000000,0.601871,0.680879,0.850234
Carambola,0.867110,0.868263,0.390736,0.920863,0.844660,0.989556,1.000000,0.979003,0.946479,0.992167
Guava,0.879822,0.741722,0.647573,0.922039,0.800256,0.897377,0.000000,0.992679,0.848521,0.945455
Kiwi,0.877707,0.977654,0.927152,0.953615,0.912987,0.954442,0.980769,0.975207,0.948454,0.974359
Mango,0.384577,0.827451,0.880240,0.676564,0.582408,0.917234,0.000000,0.950067,0.905806,0.932934
Muskmelon,0.875371,0.854962,0.911043,0.916667,0.872881,0.879147,0.050285,0.905941,0.922043,0.889688
Orange,0.738938,0.752595,0.831169,0.858881,0.598639,0.968203,0.568047,0.954617,0.908683,0.968254
Peach,0.887324,0.864919,0.837624,0.958025,0.812992,0.908766,0.000000,0.973631,0.986547,0.950758
Pear,0.933862,0.901674,0.800745,0.976804,0.789157,0.937107,0.000000,0.987226,0.883838,0.850767


In [104]:
precision_df[['Model_6', 'Model_10']]

,Model_6,Model_10
Apple,0.965398,0.914158
Banana,0.938959,0.850234
Carambola,0.989556,0.992167
Guava,0.897377,0.945455
Kiwi,0.954442,0.974359
Mango,0.917234,0.932934
Muskmelon,0.879147,0.889688
Orange,0.968203,0.968254
Peach,0.908766,0.950758
Pear,0.937107,0.850767


### Closer Look

In [106]:
model_6_predictions = model_6.predict_classes(X_rgb_test_reshape)
model_10_predictions = model_10.predict_classes(X_rgb_test_reshape)

In [107]:
model_6_precision = precision_score(y_rgb_test, model_6_predictions, average='weighted')
model_10_precision = precision_score(y_rgb_test, model_10_predictions, average='weighted')
print('Model 6', model_6_precision)
print('Model 10:', model_10_precision)

Model 6 0.9254303719434896
Model 10: 0.938281675911824


In [108]:
model_6_recall = recall_score(y_rgb_test, model_6_predictions, average='weighted')
model_10_recall = recall_score(y_rgb_test, model_10_predictions, average='weighted')
print('Model 6:', model_6_recall)
print('Model 10:', model_10_recall)

Model 6: 0.9108308939428057
Model 10: 0.9367259626210312


In [109]:
model_6_f1 = f1_score(y_rgb_test, model_6_predictions, average='weighted')
model_10_f1 = f1_score(y_rgb_test, model_10_predictions, average='weighted')
print('Model 6:', model_6_f1)
print('Model 10:', model_10_f1)

Model 6: 0.9088377725150089
Model 10: 0.9365466350483875


## Conclusion

Model 6 and Model 10 are our top 2 models based on values in validation accuracy, mse, and mae. However, I am choosing Model 10 as my main model since it has a higher precision, the main goal of my neural network. They each have very similar scores, with Model 10 doing a little better. I thought it would be interesting to take a closer look, regardless. Although Model 6 doesn't do as well as Model 10, it is providing close scores to model 10 while being a much simpler model. If the scores happened to be a little closer, I would potentially consider Model 6 to be my main model. In that situation it would have provided nearly the same information as Model 10 with less layers and without a regularizer, which could have potentially saved on time and cost.

## Packaging Predictions for EDA

In [120]:
predictions_df = pd.DataFrame(list(zip(y_rgb_test, model_10_predictions)))
predictions_df.columns = ['Actual', 'Predicted']

In [121]:
predictions_pickle_out = open('model_predictions.pickle', 'wb')
pickle.dump(predictions_df, predictions_pickle_out)
predictions_pickle_out.close()